In [2]:
from geoband.API import *

In [3]:
import pandas as pd
import folium
import geopandas as gpd
import warnings
warnings.filterwarnings(action='ignore')

In [3]:
GetCompasData('SBJ_2012_001', '2', '2.오산시_어린이교통사고_격자.geojson')
GetCompasData('SBJ_2012_001', '15', '15.오산시_무인교통단속카메라.csv')

In [4]:
data2 = gpd.read_file('2.오산시_어린이교통사고_격자.geojson')
data15 = pd.read_csv('15.오산시_무인교통단속카메라.csv')

In [5]:
data2.tail()

,gid,accident_cnt,geometry
4505,다사646052,0,"MULTIPOLYGON (((127.10138 37.14484, 127.10138 ..."
4506,다사646053,0,"MULTIPOLYGON (((127.10138 37.14575, 127.10137 ..."
4507,다사647050,0,"MULTIPOLYGON (((127.10252 37.14305, 127.10251 ..."
4508,다사647051,0,"MULTIPOLYGON (((127.10251 37.14395, 127.10251 ..."
4509,다사647052,0,"MULTIPOLYGON (((127.10251 37.14485, 127.10250 ..."


In [6]:
m = folium.Map(location = [37.149799, 127.077116], zoom_start = 12)

folium.Choropleth(geo_data = data2, data = data2, columns = ('gid', 'accident_cnt'), key_on='feature.properties.gid',
    fill_color = 'YlGnBu', legend_name='accident_cnt').add_to(m)

m.save('accident_cnt.html')

In [7]:
# 격자가 너무 작아 구별이 잘 안됨 -> 교통사고 수가 0인 곳 제외
data2.sort_values(by = 'accident_cnt', ascending = False)

data2_accident = data2[data2['accident_cnt'] != 0]

### 주정차 단속수와 교통사고수 관계###

In [9]:
GetCompasData('SBJ_2012_001', '1', '1.오산시_주정차단속(2018~2020).csv')
data1 = pd.read_csv('1.오산시_주정차단속(2018~2020).csv')

data1['area'] = ''
for idx in data1.index:
    string = data1['행정구역'][idx].split(' ')[2]
    data1['area'][idx] = string

area_df = pd.DataFrame(data1['area'].groupby(data1['area']).count())
area_df.columns = ['count']
area_df.reset_index(inplace = True)

In [10]:
GetCompasData('SBJ_2012_001', '33', '33.오산시_법정경계(읍면동).geojson')
data33 = gpd.read_file('33.오산시_법정경계(읍면동).geojson')

m = folium.Map(location = [37.149799, 127.077116], zoom_start = 12)

folium.Choropleth(geo_data = data33, data = area_df, columns = ('area', 'count'), key_on='feature.properties.EMD_KOR_NM',
    fill_color = 'YlGnBu', legend_name='주정차단속수', popup = 'area').add_to(m)

folium.Choropleth(geo_data = data2_accident, data = data2_accident, columns = ('gid', 'accident_cnt'), key_on='feature.properties.gid',
    fill_color = 'PuRd', legend_name='교통사고수').add_to(m)

m.save('주정차 단속수 & 교통사고 수.html')

### 주정차단속CCTV수와 교통사고수의 관계###

In [11]:
from folium.plugins import MarkerCluster

m = folium.Map(location = [37.149799, 127.077116], zoom_start = 12)

folium.Choropleth(geo_data = data2_accident, data = data2_accident, columns = ('gid', 'accident_cnt'), key_on='feature.properties.gid',
    fill_color = 'PuRd', legend_name='교통사고수').add_to(m)

marker_cluster = MarkerCluster().add_to(m) 
for idx in data15.index: 
    folium.Marker(location = [data15['설치위치_위도'][idx],data15['설치위치_경도'][idx]], 
                  icon=folium.Icon(icon = 'camera') ).add_to(marker_cluster)

m.save('CCTV수 & 교통사고수.html')

# 세교동 : 도로에 교통사고 수가 많은 것으로 보아 주정차보다는 다른 요인에 의해 교통사고가 자주 일어나는 듯해보임
# 가수동 : 오산초 쪽 CCTV 늘릴 필요가 있어보임.

### CCTV(C,E) 와 교통사고수관계 ###

In [12]:
GetCompasData('SBJ_2012_001', '20', '20.오산시_CCTV설치현황.csv')
data20 = pd.read_csv('20.오산시_CCTV설치현황.csv')

In [13]:
data20_C = data20[data20['CCTV 유형코드'] == 'C']
data20_E = data20[data20['CCTV 유형코드'] == 'E']

data20_C.replace(172.058905, 127.058905, inplace = True)
data20_E.rename(columns = {'설치위치_위도':'설치위치_경도', '설치위치_경도':'설치위치_위도'}, inplace = True)

In [14]:
m = folium.Map(location = [37.149799, 127.077116], zoom_start = 13)

folium.Choropleth(geo_data = data2_accident, data = data2_accident, columns = ('gid', 'accident_cnt'), key_on='feature.properties.gid',
    fill_color = 'PuRd', legend_name='교통사고수').add_to(m)

for idx in data20_C.index:
    folium.Marker([data20_C['설치위치_위도'][idx],data20_C['설치위치_경도'][idx]], 
                  popup = data20_C['설치위치_도로명주소'][idx], icon = folium.Icon(icon = 'camera',color = 'blue')).add_to(m)

for idx in data20_E.index:
    folium.Marker([data20_E['설치위치_위도'][idx],data20_E['설치위치_경도'][idx]], 
                  popup = data20_E['설치위치_도로명주소'][idx], icon = folium.Icon(icon = 'camera',color = 'red')).add_to(m)

m.save('CCTV(C,E) & 교통사고수.html')
    
# 교육지원청 사거리, 한정 사거리, 오산시청 앞 사거리, 원동사거리 과속단속 CCTV 설치 필요성?